In [ ]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain langchain_fireworks google-search-results requests gradio

In [2]:
import os
os.environ['FIREWORKS_API_KEY'] = 'API_KEY'
os.environ["SERPER_API_KEY"] = 'API_KEY'

In [12]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain_fireworks import ChatFireworks
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
import gradio as gr
import os


# Set up the prompt template
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a chatbot having a conversation with a human. You might have multiple sessions but you can use previous chats if necessary."),
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("{human_input}")
])

# Initialize the language model
llm = ChatFireworks(model_name="accounts/fireworks/models/llama-v3p1-70b-instruct", temperature=0.3)

# Initialize memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Create the conversation chain
chat_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

def chat(message, history):
    response = chat_chain.predict(human_input=message)
    history.append((message, response))
    return "", history

def new_chat():
    chat_chain.predict(human_input="[NEW CHAT SESSION]")
    return None, []

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("New Chat")

    msg.submit(chat, [msg, chatbot], [msg, chatbot])
    clear.click(new_chat, outputs=[msg, chatbot])

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://303d74ff7cdb6fbfee.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
